# Temperature and metallic systems

In this example we consider the modeling of a magnesium lattice
as a simple example for a metallic system.
For our treatment we will use the PBE exchange-correlation functional.
First we import required packages and setup the lattice.
Again notice that DFTK uses the convention that lattice vectors are
specified column by column.

In [1]:
using DFTK
using Plots

a = 3.01794  # bohr
b = 5.22722  # bohr
c = 9.77362  # bohr
lattice = [[-a -a  0]; [-b  b  0]; [0   0 -c]]
Mg = ElementPsp(:Mg, psp=load_psp("hgh/pbe/Mg-q2"))
atoms = [Mg => [[2/3, 1/3, 1/4], [1/3, 2/3, 3/4]]];

┌ Warning: Package DFTK does not have Plots in its dependencies:
│ - If you have DFTK checked out for development and have
│   added Plots as a dependency but haven't updated your primary
│   environment's manifest file, try `Pkg.resolve()`.
│ - Otherwise you may need to report an issue with DFTK
│ Loading Plots into DFTK from project dependency, future warnings for DFTK are suppressed.
└ @ nothing nothing:910


Next we build the PBE model and discretize it.
Since magnesium is a metal we apply a small smearing
temperature to ease convergence using the Fermi-Dirac
smearing scheme. Note that both the `Ecut` is too small
as well as the minimal ``k``-point spacing
`kspacing` far too large to give a converged result.
These have been selected to obtain a fast execution time.
By default `PlaneWaveBasis` chooses a `kspacing`
of `2π * 0.022` inverse Bohrs, which is much more reasonable.

In [2]:
kspacing = 0.5      # Minimal spacing of k-points,
#                    in units of wavevectors (inverse Bohrs)
Ecut = 5            # kinetic energy cutoff in Hartree
temperature = 0.01  # Smearing temperature in Hartree

model = model_DFT(lattice, atoms, [:gga_x_pbe, :gga_c_pbe];
                  temperature=temperature,
                  smearing=DFTK.Smearing.FermiDirac())
kgrid = kgrid_size_from_minimal_spacing(lattice, kspacing)
basis = PlaneWaveBasis(model, Ecut, kgrid=kgrid);

Finally we run the SCF. Two magnesium atoms in
our pseudopotential model result in four valence electrons being explicitly
treated. Nevertheless this SCF will solve for eight bands by default
in order to capture partial occupations beyond the Fermi level due to
the employed smearing scheme.

In [3]:
scfres = self_consistent_field(basis);

n     Free energy       Eₙ-Eₙ₋₁     ρout-ρin   Diag
---   ---------------   ---------   --------   ----
  1   -1.761885130995         NaN   5.06e-02    4.0 
  2   -1.762195714381   -3.11e-04   9.80e-03    1.0 
  3   -1.762239577537   -4.39e-05   6.75e-04    3.7 
  4   -1.762241611402   -2.03e-06   1.10e-04    3.7 
  5   -1.762241620371   -8.97e-09   1.45e-05    2.3 


In [4]:
scfres.occupation[1]

9-element Array{Float64,1}:
 1.999999999907795
 1.999997586292215
 0.004016900144344562
 3.000560253486482e-15
 1.1115593634001013e-18
 1.1114803183320936e-18
 7.977900449769684e-19
 7.977287317129825e-19
 3.344321794470875e-22

In [5]:
scfres.energies

Energy breakdown:
    Kinetic             0.7180655 
    AtomicLocal         0.3145397 
    AtomicNonlocal      0.3265742 
    Ewald               -2.1544222
    PspCorrection       -0.1026056
    Hartree             0.0055004 
    Xc                  -0.8610491
    Entropy             -0.0088446

    total               -1.762241620371


The fact that magnesium is a metal is confirmed
by plotting the density of states around the Fermi level.

In [6]:
plot_dos(scfres)